In [14]:
import mne
import numpy as np

In [20]:
file = "C:\\Users\\Luchitha\Documents\\BCI\BCI_IV\\dataset\\A01T.gdf"

In [21]:

raw = mne.io.read_raw_gdf(file, preload=True)



Extracting GDF parameters from C:\Users\Luchitha\Documents\BCI\BCI_IV\dataset\A01T.gdf...
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


C:\Users\Luchitha\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


In [22]:
raw.drop_channels(['EOG-left', 'EOG-central', 'EOG-right'])


<RawGDF | A01T.gdf, 22 x 672528 (2690.1 s), ~112.9 MiB, data loaded>

In [25]:
raw.filter(8, 30)  # Motor imagery typically occurs in the mu (8-12 Hz) and beta (13-30 Hz) frequency bands


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation


- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)



<RawGDF | A01T.gdf, 22 x 672528 (2690.1 s), ~112.9 MiB, data loaded>

Important events:

769 → left

770 → right

771 → feet

772 → tongue

Epoch window :

2.0 s → 6.0 s after cue

In [26]:


events, event_id = mne.events_from_annotations(raw)
print(event_id)




Used Annotations descriptions: [np.str_('1023'), np.str_('1072'), np.str_('276'), np.str_('277'), np.str_('32766'), np.str_('768'), np.str_('769'), np.str_('770'), np.str_('771'), np.str_('772')]
{np.str_('1023'): 1, np.str_('1072'): 2, np.str_('276'): 3, np.str_('277'): 4, np.str_('32766'): 5, np.str_('768'): 6, np.str_('769'): 7, np.str_('770'): 8, np.str_('771'): 9, np.str_('772'): 10}


In [28]:
epochs = mne.Epochs(
    raw,
    events,
    event_id={
        'LH': event_id['769'],
        'RH': event_id['770'],
        'F':  event_id['771'],
        'T':  event_id['772'],
    },
    tmin=2.0,
    tmax=6.0,
    baseline=None,
    preload=True
)


Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 1001 original time points ...
1 bad epochs dropped


In [29]:
print(epochs)
print(epochs.event_id)


<Epochs | 287 events (all good), 2 – 6 s (baseline off), ~48.2 MiB, data loaded,
 'LH': 71
 'RH': 72
 'F': 72
 'T': 72>
{'LH': 7, 'RH': 8, 'F': 9, 'T': 10}
